# Generate ELK Snapshot on Meas Node

### This notebook creates an ELK snapshot dir on the measurement node and prepares a .tar file to be exported

## Import

In [ ]:
import os
import json
import traceback
import datetime
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

In [ ]:
import importlib
from mflib.mf_data_transfer import mf_data_export

## Slice Info

In [ ]:
# Change your slice name and node name
slice_name = "MyMonitoredSlice"
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

In [ ]:
mf = MFLib(slice_name)

In [ ]:
meas_node_name = mf.measurement_node_name
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    meas_node = slice.get_node(name=meas_node_name)
except Exception as e:
    print(f"Fail: {e}")   
#print (meas_node)

## Create the data export object

In [ ]:
mde = mf_data_export(slice_name=slice_name, meas_node_name=meas_node_name)

## Create ELK Snapshot

### Change directory permission

In [ ]:
# This will change the permission of /var/lib/docker/volumes/elk_snapshotbackup/_data for elk to write snapshot to
mde.change_elk_dir_permission(node=mf.measurement_node_name)

### Create a snapshot repository

In [ ]:
mde.create_snapshot_repo(repo_name="backup_repository")

### Check the created repo

In [ ]:
mde.verify_repo(repo_name="backup_repository")

### List indice

In [ ]:
mde.list_indices()

### Create snapshot

In [ ]:
# Default will include all indices. add arg (list) indice=[index1, index2] to specify indice
# May take a while
mde.create_elk_snapshot(repo_name="backup_repository", snapshot_name="snapshot_test")

## Create dir for the elk tar file

In [ ]:
elk_tar_dir = "/home/mfuser/services/elk/files/snapshots"
mde.create_dir_for_elk_tar(dir_name=elk_tar_dir)

## Make the snapshot tar file

In [ ]:
#snapshot_name = "elk_backup_" + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
snapshot_name="snapshot_test"
mde.tar_elk_snapshot(dir_name=elk_tar_dir, file_name=snapshot_name)

### Download index from elk directory to local storage

In [ ]:
# files = [snapshot_name + ".tar"]
files = ["snapshot_test.tar"]

# Specify output location (Default is current directory)
outputDirectory = "./snapshots/"
stdout = []
for file in files:
    stdout.append(mf.download_service_file("elk", "files/snapshots/" + file , outputDirectory + file))

## Export complete
The file is located in your current directories snapshots directory.

You can now download this file to import into another elk stack.

For import instructions, refer back to [Start here](.start_here.ipynb)

## OPTIONAL
Delete the snapshot and repository if you are done

Do **NOT** run the following if you want to restore the snapshot on measurement node since it can be achieved using a single curl command

### Delete the snapshot

In [ ]:
#mde.delete_snapshot(repo_name="backup_repository", snapshot_name="snapshot_test")

### Delete the repository

In [ ]:
#mde.delete_repo(repo_name="backup_repository")